In [2]:
import random
import algviz
# Create a visualizer object.
# The default duration of each frame of animation is 1.0 second.
# The interval between two frames of animation is 0.5 seconds.
viz = algviz.Visualizer(delay=1.0, wait=0.5)

viz.display()               # Display the firt animation frame.
# Do something...
viz.display(delay=1.0)      # Display the second animation frame.

tab = viz.createTable(
    row=1,                    # The table has 3 rows.
    col=1,                    # The table has 3 columns
    data=None,                # The initial data in table.
    name="Table",             # The name of table.
    cell_size=(40, 40),       # The table cell (width, height).
    show_index=True           # Whether to display subscripts.
)
viz.display()

curLength = 0                   # Keeps track of the length of the skiplist (number of cols)
maxDepth = 3                    # Sets the maximum number of rows
curAction = '_placeholder_'     # Will become insert, delete, search, or done
while curAction != 'done':
    curAction = input('Enter \'insert\', \'delete\', \'search\', or \'done\':' )

    if curAction == 'insert':
        print('You said insert...')
        curInput = input('Enter an integer: ')  # String value from user input
        curInt = -1                             # Will be replaced with the value of the integer, if input is valid
        try:                                    # Make sure input is valid. Otherwise, start over.
        # Convert it into integer
            curInt = int(curInput)
        except ValueError:
            print('\'', curInput, '\' is not a valid integer...')
            continue
        print('time to add: ', curInt)
        if curLength == 0:
            tab.reshape(row=maxDepth, col=1)
            tab[maxDepth-1][0] = curInt
            curDepth = 1
            nextLevel = random.choice([0,1])==1 # boolean value. 50% chance we go up a level.
            while curDepth+1 is not maxDepth:
                viz.display(0.5)
            

    elif curAction == 'delete':
        print('You said delete...')
    elif curAction == 'search':
        print('You said search...')
    elif curAction == 'done':
        print('You said done...')
    else:
        print("INVALID INPUT! Try again...")

IndentationError: expected an indented block after 'while' statement on line 44 (2883392896.py, line 47)